# Military Emissions

This notebook is responsible for working with data from [The Military Emissions Gap](https://militaryemissions.org/) and uses data countries reported to the UN Framework Convention on Climate Change. However, reporting for emissions from military is voluntary.


### Excerpt from The Military Emissions Gap
Military GHG emissions data reported by Parties to the UNFCCC has been examined to gauge its status, transparency and consistency.

Under guidelines published by the Intergovernmental Panel on Climate Change (IPCC), military fuel use should be reported under IPCC category 1.A.5 (Other, not elsewhere specified). This category includes all mobile fuel consumption, such as ships, aircraft and road vehicles. It also includes all stationary fuel consumption, such as heating buildings on military bases.

GHG emissions should be reported at the most disaggregated level of each source, but current guidelines allow aggregation to protect military information. But if they are aggregated, this means that emissions from military mobile and stationary fuel use cannot be properly distinguished in UNFCCC reporting.

In [1]:
import pandas as pd

In [2]:
# Read the CSV file into a dataframe
df = pd.read_csv('data/military_emissions.csv')
df.drop(['In country reporting', 'Latest national inventory report (NIR)', 'Link  to  report'], axis=1, inplace=True)
# Display the head of the dataframe
df.head()

,Country name,Stationary emissions (as reported under 1A5a) MtCO2e,Mobile emissions (as reported under 1A5b) MtCO2e,Gap in emissions reporting,Data accessibility score,"Military expenditure (2022), US$ millions",GDP - US$ millions,Global military expenditure rank,% of GDP
0,Australia,"""Not occurring"".",0.816,Very significant gap in reporting.,Poor - disaggregated data not reported through...,"32,299.00","1,675,418.67",13,1.93
1,Austria,"""Not occurring"".",0.030,Very significant gap in reporting.,Poor - disaggregated data not reported through...,"3,626.00","471,400.07",50,0.77
2,Belarus,1.045,"""Included elsewhere"".",No comparison possible.,Poor - disaggregated data not reported through...,821.00,"72,793.46",80,1.13
3,Belgium,"""Not occurring"".",0.105,Very significant gap in reporting.,Poor - disaggregated data not reported through...,"6,867.00","578,604.10",33,1.19
4,Bulgaria,"""Not occurring"".",0.001,Very significant gap in reporting.,Poor - disaggregated data not reported through...,"1,336.00","89,040.40",68,1.50


In [3]:
from functions import country_to_iso
    
df['ISO_country'] = df['Country name'].apply(country_to_iso)

In [4]:
from functions import df_to_database

print(df_to_database(df, 'militaryEmissions'))

Success.


In [5]:
df['Data accessibility score'] = df['Data accessibility score'].replace({
    'Very poor - no data reported through the UNFCCC.': 'Very Poor',
    'Poor - disaggregated data not reported through the UNFCCC.': 'Poor',
    'Fair - data reported through the UNFCCC.': 'Fair',
    'Poor - submitted data through the UNFCCC includes civilian sources.': 'Poor',
    'Poor - submitted data through the UNFCCC includes other civilian sources.': 'Poor',
    'Poor - submitted data includes civilian sources.': 'Poor',
    'Poor - data submitted under 1.A.5 includes civilian sources and overall military fuel data reported through the UNFCCC is not clearly disaggregated.': 'Poor',
})
print(df['Data accessibility score'].value_counts())
fair_countries = df[df['Data accessibility score'] == 'Fair']['Country name']
print(fair_countries)

Data accessibility score
Poor         28
Very Poor     8
Fair          4
Name: count, dtype: int64
7       Cyprus
13     Germany
15     Hungary
31    Slovakia
Name: Country name, dtype: object


In [6]:
accessibilityDf = df['Data accessibility score'].value_counts()
accessibilityDf.loc['Good'] = 0
accessibilityDf['No data'] = 198 - accessibilityDf.sum() # 198 parties to UNFCCC
accessibilityDf = accessibilityDf.reindex(index=['Very Poor', 'Poor', 'Fair', 'Good', 'No data'])

accessibilityDf.head()
accessibilityDf.sum()

198

In [7]:
import plotly.express as px
filtered_df = accessibilityDf[accessibilityDf.index != 'No data']
fig = px.bar(filtered_df, x=filtered_df.index, y=filtered_df.values)
fig.update_layout(
    xaxis_title='Score',
    yaxis_title='Number of Countries',
)
fig.show()

In [8]:
fig = px.bar(accessibilityDf, x=accessibilityDf.index, y=accessibilityDf.values)
fig.update_layout(
    xaxis_title='Score',
    yaxis_title='Number of Countries',
)
fig.show()